## 데이터 실습

### 1.데이터 불러오기 

In [16]:
import numpy as np
import pandas as pd

In [17]:
train = pd.read_csv("./data/train.csv",index_col='PassengerId')
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 2. 데이터 확인하기
    - shape
    - info
    - 결측치

In [20]:
train.shape

(891, 11)

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


- `age`,`cabin`,`Embarked`에 결측치 존재
  - 어떻게 처리할것인가? 를 고민해볼 수 있음
- `name`,`Sex`,`Ticket`,`Cabin`,`Embarked` > string type

In [35]:
train['Age'].isnull().sum() #결측치 개수 확인

np.int64(177)

In [38]:
train['Cabin'].isnull().sum()

np.int64(687)

### 3. 전처리 응용실습 

#### 3.1 Age
- Age 결측치를 전체 평균 나이값으로 채우기;

In [51]:
train['Age'].fillna(train['Age'].mean().round())

PassengerId
1      22.0
2      38.0
3      26.0
4      35.0
5      35.0
       ... 
887    27.0
888    19.0
889    30.0
890    26.0
891    32.0
Name: Age, Length: 891, dtype: float64

In [50]:
train['Age'].mean().round()

np.float64(30.0)

In [54]:
train['Age'][train['Age'].isna()==False].mean().round()

np.float64(30.0)

In [55]:
train['Age'].mean().astype(int) # astype함수를 통해 int형으로 바꿔서 넣어도 된다.

np.int64(29)

##### transform 함수
- groupby와 함께 사용

- Age 결측치를 성별 평균 or 중간값으로 채우기

In [64]:
train[['Age','Sex']].groupby('Sex').mean()

,Age
Sex,
female,27.915709
male,30.726645


- 위의 방법 : 데이터프레임에 groupby 함수를 적용

In [65]:
train.groupby('Sex')['Age'].mean() # 이렇게 작성해도 됨

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

- 전체 데이터셋에 groupby함수를 적용 > Series 데이터 형태로 나옴

```
전체데이터.groupby('기준명')['적용 컬럼'].함수명()
```

In [66]:
train[['Age','Sex']].groupby('Sex').median()

,Age
Sex,
female,27.0
male,29.0


- 기준 컬럼명이 -> Age가 아니라 성별이므로 "Sex"

In [71]:
train.groupby('Sex')['Age'].transform('median')

PassengerId
1      29.0
2      27.0
3      27.0
4      27.0
5      29.0
       ... 
887    29.0
888    27.0
889    27.0
890    29.0
891    29.0
Name: Age, Length: 891, dtype: float64

- 전체 데이터를 각 성별의 중앙값 으로 반영 > 그러면 fillna를 통해 결측치만 해당 값으로 바꿔주면 됨

In [73]:
train['Age'].fillna(train.groupby('Sex')['Age'].transform('median'),inplace = True)

In [75]:
train['Age'].isna().sum()

np.int64(0)

#### 3.2 Embarked 결측치 채우기
- Embarked 결측치를 최빈값으로 채우기;

In [77]:
train['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [80]:
train['Embarked'].isnull().sum()

np.int64(2)

In [105]:
train['Embarked'].value_counts().index[0]

'S'

- 값이 업데이트 될때마다 새로 찾아서 넣는 것보다 코드 자체로 넣어주기

In [114]:
train['Embarked'].fillna(train['Embarked'].value_counts().index[0],inplace = True)

- 또다른 방법

In [108]:
train['Embarked'].value_counts().idxmax() # value_counts 중 가장 큰 인덱스이름 가져오기

'S'

### 4.데이터 분석

#### 4.1 성별에 따른 생존자/사망자 비율

----

- 처음 내가 짠 코드

In [132]:
train['Sex'][train['Survived']==0].count()

np.int64(549)

In [134]:
train['Sex'][train['Survived']==0].value_counts()

Sex
male      468
female     81
Name: count, dtype: int64

-------

- 전체 사망자 중의 남녀 비율

In [128]:
train['Sex'][train['Survived']==0].value_counts()/train['Sex'][train['Survived']==0].count()*100

Sex
male      85.245902
female    14.754098
Name: count, dtype: float64

- 전체 생존자 중의 남녀 비율

In [127]:
train['Sex'][train['Survived']==1].value_counts()/train['Sex'][train['Survived']==1].count()*100

Sex
female    68.128655
male      31.871345
Name: count, dtype: float64

------

- 전체 남성의 생존자와 사망자비율 > 내가 생각했던 코드

In [141]:
train['Survived'][train['Sex']=='female'].value_counts()/train['Survived'][train['Sex']=='female'].count()*100

Survived
1    74.203822
0    25.796178
Name: count, dtype: float64

In [146]:
train['Survived'][train['Sex']=='female'].value_counts(1)

Survived
1    0.742038
0    0.257962
Name: proportion, dtype: float64

- 전체 여성의 생존자와 사망자비율

In [140]:
train['Survived'][train['Sex']=='male'].value_counts()/train['Survived'][train['Sex']=='male'].count()*100

Survived
0    81.109185
1    18.890815
Name: count, dtype: float64

In [147]:
train['Survived'][train['Sex']=='male'].value_counts(1)

Survived
0    0.811092
1    0.188908
Name: proportion, dtype: float64

-------

- groupby 사용 ver1

In [144]:
train.groupby('Sex')['Survived'].mean().round(3)

Sex
female    0.742
male      0.189
Name: Survived, dtype: float64

In [145]:
1-train.groupby('Sex')['Survived'].mean().round(3)

Sex
female    0.258
male      0.811
Name: Survived, dtype: float64

---

- groupby 사용 ver2

In [149]:
train.groupby('Sex')['Survived'].value_counts(1)*100

Sex     Survived
female  1           74.203822
        0           25.796178
male    0           81.109185
        1           18.890815
Name: proportion, dtype: float64

-------

#### 4.2 나이대별 생존자/사망자 비율

In [159]:
train['Age'].isnull().sum()

np.int64(0)

In [150]:
train['Age'].min()

0.42

In [151]:
train['Age'].max()

80.0

In [152]:
labels = ['0대','10대','20대','30대','40대','50대','60대','70대','80대']
bins = [-1,9,19,29,39,49,59,69,79,89]
dt = train['Age']

In [156]:
train.insert(loc = 5,column = 'Age_cat',value = pd.cut(dt,bins,labels=labels))

In [157]:
train

,Survived,Pclass,Name,Sex,Age,Age_cat,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,20대,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,30대,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,20대,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,30대,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,30대,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,20대,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,10대,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,27.0,20대,1,2,W./C. 6607,23.4500,NaN,S


In [174]:
train.groupby(['Age_cat'])['Survived'].value_counts(1)*100

C:\Users\ChoiSeokjin\AppData\Local\Temp\ipykernel_2080\3373111959.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby(['Age_cat'])['Survived'].value_counts(1)*100


Age_cat  Survived
0대       1            61.290323
         0            38.709677
10대      0            59.803922
         1            40.196078
20대      0            67.506297
         1            32.493703
30대      0            56.287425
         1            43.712575
40대      0            61.797753
         1            38.202247
50대      0            58.333333
         1            41.666667
60대      0            68.421053
         1            31.578947
70대      0           100.000000
         1             0.000000
80대      1           100.000000
         0             0.000000
Name: proportion, dtype: float64

In [179]:
train.groupby(['Age_cat'])['Survived'].value_counts()/train.groupby(['Age_cat'])['Survived'].count()*100

C:\Users\ChoiSeokjin\AppData\Local\Temp\ipykernel_2080\3770204234.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby(['Age_cat'])['Survived'].value_counts()/train.groupby(['Age_cat'])['Survived'].count()*100
C:\Users\ChoiSeokjin\AppData\Local\Temp\ipykernel_2080\3770204234.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby(['Age_cat'])['Survived'].value_counts()/train.groupby(['Age_cat'])['Survived'].count()*100


Age_cat  Survived
0대       1            61.290323
         0            38.709677
10대      0            59.803922
         1            40.196078
20대      0            67.506297
         1            32.493703
30대      0            56.287425
         1            43.712575
40대      0            61.797753
         1            38.202247
50대      0            58.333333
         1            41.666667
60대      0            68.421053
         1            31.578947
70대      0           100.000000
         1             0.000000
80대      1           100.000000
         0             0.000000
dtype: float64

In [182]:
train[['Survived','Age_cat']].groupby('Age_cat').value_counts()

C:\Users\ChoiSeokjin\AppData\Local\Temp\ipykernel_2080\2682988861.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train[['Survived','Age_cat']].groupby('Age_cat').value_counts()


Age_cat  Survived
0대       1            38
         0            24
10대      0            61
         1            41
20대      0           268
         1           129
30대      0            94
         1            73
40대      0            55
         1            34
50대      0            28
         1            20
60대      0            13
         1             6
70대      0             6
         1             0
80대      1             1
         0             0
Name: count, dtype: int64

- groupby에 카테고리를 두개를 같이 넣으면 각 카테고리별로 groupby를 적용

In [188]:
train.groupby(['Sex','Age_cat'])['Survived'].value_counts()

C:\Users\ChoiSeokjin\AppData\Local\Temp\ipykernel_2080\2564392976.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train.groupby(['Sex','Age_cat'])['Survived'].value_counts()


Sex     Age_cat  Survived
female  0대       1            19
                 0            11
        10대      1            34
                 0            11
        20대      1            88
                 0            37
        30대      1            50
                 0            10
        40대      1            22
                 0            10
        50대      1            16
                 0             2
        60대      1             4
                 0             0
        70대      0             0
                 1             0
        80대      0             0
                 1             0
male    0대       1            19
                 0            13
        10대      0            50
                 1             7
        20대      0           231
                 1            41
        30대      0            84
                 1            23
        40대      0            45
                 1            12
        50대      0            26
                 

In [166]:
train[train['Age_cat']=="70대"]

,Survived,Pclass,Name,Sex,Age,Age_cat,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,,
97,0,1,"Goldschmidt, Mr. George B",male,71.0,70대,0,0,PC 17754,34.6542,A5,C
117,0,3,"Connors, Mr. Patrick",male,70.5,70대,0,0,370369,7.7500,NaN,Q
494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,70대,0,0,PC 17609,49.5042,NaN,C
673,0,2,"Mitchell, Mr. Henry Michael",male,70.0,70대,0,0,C.A. 24580,10.5000,NaN,S
746,0,1,"Crosby, Capt. Edward Gifford",male,70.0,70대,1,1,WE/P 5735,71.0000,B22,S
852,0,3,"Svensson, Mr. Johan",male,74.0,70대,0,0,347060,7.7750,NaN,S


In [168]:
train[train['Age']>=70 |train['Age']<80] # 묶어주지 않으면 비교 연산자 에러

TypeError: Cannot perform 'ror_' with a dtyped [float64] array and scalar of type [bool]

In [173]:
train[(train['Age']>=70 ) & (train['Age']<80)] 

,Survived,Pclass,Name,Sex,Age,Age_cat,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,,
97,0,1,"Goldschmidt, Mr. George B",male,71.0,70대,0,0,PC 17754,34.6542,A5,C
117,0,3,"Connors, Mr. Patrick",male,70.5,70대,0,0,370369,7.7500,NaN,Q
494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,70대,0,0,PC 17609,49.5042,NaN,C
673,0,2,"Mitchell, Mr. Henry Michael",male,70.0,70대,0,0,C.A. 24580,10.5000,NaN,S
746,0,1,"Crosby, Capt. Edward Gifford",male,70.0,70대,1,1,WE/P 5735,71.0000,B22,S
852,0,3,"Svensson, Mr. Johan",male,74.0,70대,0,0,347060,7.7750,NaN,S


In [ ]:
train['Sex'][train['Survived']==0].value_counts()/train['Sex'][train['Survived']==0].count()*100

In [135]:
train[['Survived','Sex']].value_counts()

Survived  Sex   
0         male      468
1         female    233
          male      109
0         female     81
Name: count, dtype: int64

In [137]:
train['Survived'].count()

np.int64(891)

In [139]:
train[['Survived','Sex']].value_counts()/train['Survived'].count()*100

Survived  Sex   
0         male      52.525253
1         female    26.150393
          male      12.233446
0         female     9.090909
Name: count, dtype: float64

- 이렇게 구하면 모집단이 전체로 값들을 나눠서 계산했던 거임

In [39]:
train['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [37]:
train['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [40]:
train['Ticket'].value_counts()

Ticket
347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 681, dtype: int64

--------


#### 개인연습

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


- *결측치가 많은 컬럼은 어떻게 할 것인가?, Age가 실수값이면 어떻게 처리할 것인가?, 이상치는 어떻게 처리할 것인가?*
  > 고민해보기

In [7]:
train.describe() # 데이터의 분위수 확인

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### Embarked 결측치 채우기 
- 다른 방법이 없을까? 다른 값들이 유사한 사람들과 같은 값을 채우려고

In [92]:
train[train['Embarked'].isnull()==True]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


- 헤멨던 이유
  - train자체로 인덱싱을 해야하는데 train['Embarked']로 인덱싱을 하고 있었음...
  - `train['Embarked'].isnull()==True` 로 True값을 찾아야하는데 `train['Embarked']==True`로 찾고 있엇음

In [96]:
train['Ticket'].value_counts()

Ticket
347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 681, dtype: int64

In [99]:
train[train['Ticket']=='113572']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [102]:
train[train['Cabin']=='B28']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [103]:
train[train['Fare']==80.0]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


- 이 두사람은 다른 사람들과 공통점을 찾을 수 가 없어서 그냥 최빈값을 채우는 게 좋을 것 같음!

In [113]:
train[train['Age']==38.0]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,NaN,S
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN
109,0,3,"Rekic, Mr. Tido",male,38.0,0,0,349249,7.8958,NaN,S
225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0000,C93,S
333,0,1,"Graham, Mr. George Edward",male,38.0,0,1,PC 17582,153.4625,C91,S
358,0,2,"Funk, Miss. Annie Clemmer",female,38.0,0,0,237671,13.0000,NaN,S
466,0,3,"Goncalves, Mr. Manuel Estanslas",male,38.0,0,0,SOTON/O.Q. 3101306,7.0500,NaN,S
472,0,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S
